In [20]:
import pandas as pd
import sqlite3


In [21]:
conn = sqlite3.connect(':memory:')

In [22]:
employee = pd.DataFrame({'id': [1,2,3,4,5,6,7],
                   'name': ['Joe', 'Henry', 'Sam', 'Max', 'Janet', 'Randy', 'Will'],
                   'salary' : [85000, 80000, 60000, 90000, 69000, 85000, 70000],
                    'departmentId' : [1,2,2,1,1,1,1]
                  })

In [27]:
dept = pd.DataFrame({'id': [1,2],
                   'name': ['IT', 'Sales']
                  })

In [26]:
employee.to_sql('employee', con=conn, index=False, if_exists='replace')

7

In [28]:
dept.to_sql('department', con=conn, index=False, if_exists='replace')

2

Find the highest earning employee under each department

In [43]:
query = """

    with top_salary as (select id,name, departmentID, salary,
    dense_rank() over(partition by departmentID order by salary desc) as salary_rank
    from employee
    )

    select ts.id as emp_id, ts.name as emp_name , ts.salary, d.name as department_name from top_salary ts
    inner join department d
    on d.id = ts.departmentId
    where salary_rank <= 3 
    order by d.name , ts.salary desc
"""

In [44]:
pd.read_sql(query, con=conn)

,emp_id,emp_name,salary,department_name
0,4,Max,90000,IT
1,1,Joe,85000,IT
2,6,Randy,85000,IT
3,7,Will,70000,IT
4,2,Henry,80000,Sales
5,3,Sam,60000,Sales


In [48]:
group_1 = pd.read_csv('Group_1.csv')
group_2 = pd.read_csv('Group_2.csv')
group_2

,PersonID,Height_CM,DOB
0,14521,166,1991-3-9
1,15121,171,1989-2-12
2,21824,175,1993-10-11
3,39721,169,1981-2-12
4,42291,167,1982-3-2
5,11931,163,1984-4-12
6,14123,162,1986-6-10


In [47]:
group_1.to_sql('group_1',con=conn, index=False, if_exists='replace')
group_2.to_sql('group_2', con=conn, index=False, if_exists='replace')

7

In [53]:
query = """

with group_1_tallest as (
    select personid, cast(Height_CM as decimal) as height,DOB,'Tallest_g1' as id, rank() over(order by Height_CM desc) as ranked
    from group_1
    ),
    group_1_eldest as (
    select personid, cast(Height_CM as decimal) as height,DOB, 'eldest_g1' as id,rank() over(order by DOB ) as ranked
    from group_1
    ),
    group_2_tallest as (
    select personid, cast(Height_CM as decimal) as height,DOB, 'Tallest_g2' as id,rank() over(order by Height_CM desc) as ranked
    from group_2
    ),
    group_2_eldest as (
    select personid, cast(Height_CM as decimal) as height,DOB, 'eldest_g2' as id,rank() over(order by DOB ) as ranked
    from group_2
    )
select personid, height, DOB , id from group_1_tallest where ranked =1
union 
select personid, height, DOB , id from group_1_eldest where ranked =1
union
select personid, height, DOB , id from group_2_tallest where ranked =1
union
select personid, height, DOB , id from group_2_eldest where ranked =1

"""

In [54]:
pd.read_sql(query, con=conn)

,personid,height,DOB,id
0,14521,160,1963-9-3,eldest_g1
1,21821,175,1976-10-11,Tallest_g1
2,21824,175,1993-10-11,Tallest_g2
3,39721,169,1981-2-12,eldest_g2


In [55]:
product_spend_df = pd.DataFrame({'category': ['appliance','appliance','appliance','electronics','electronics','electronics'],
                    'product': ['refrigerator','refrigerator','washing machine','vacuum', 'wireless headset', 'vacuum'],
                   'user_id' : [165,123,123,178,156,145],
                    'spend' : [246,299.99,219.8,152,249.9,189],
                    'transaction_date' : ['12/26/2022','3/2/2022','3/2/2022','4/5/2022','7/8/2022', '7/15/2022']
                  })

Rank the products in each category in descending order based on the total amount spent by consumers to acquire them. the product with the highest spend should come first, followed by the others in each category.

In [56]:
product_spend_df.to_sql('product', con=conn, index=False)

6

In [79]:
query = """

with prod_spend as (
    select category , product , sum(spend) as amount_spent 
    from product group by category , product),
    prod_spend_rank as (
    select *, dense_rank() over(partition by category order by amount_spent desc) as spend_rank from prod_spend)


select * from prod_spend_rank
order by category, spend_rank
"""

In [80]:
pd.read_sql(query, con=conn)

,category,product,amount_spent,spend_rank
0,appliance,refrigerator,545.99,1
1,appliance,washing machine,219.80,2
2,electronics,vacuum,341.00,1
3,electronics,wireless headset,249.90,2


In [81]:
ticker = pd.read_csv("./Ticker.csv")

In [82]:
ticker.to_sql('ticker', con=conn, index=False)

756

In [83]:
ticker.head()

,Date,Close,Ticker
0,2021-08-20,88.940002,ORCL
1,2021-08-23,89.120003,ORCL
2,2021-08-24,88.550003,ORCL
3,2021-08-25,88.620003,ORCL
4,2021-08-26,88.720001,ORCL


### Max close price for each ticker
Calculate the maximum close price for each ticker


In [94]:
query = """

with max_close_rate as (
    select ticker , date, close as closing_rate, max(close) over(partition by ticker order by date(Date)) as max_closing_rate 
    from ticker)
select * from max_close_rate
order by ticker, date
"""

In [95]:
pd.read_sql(query, con=conn)

,ticker,date,closing_rate,max_closing_rate
0,MAZN,2021-08-20,159.997498,159.997498
1,MAZN,2021-08-23,163.293503,163.293503
2,MAZN,2021-08-24,165.289001,165.289001
3,MAZN,2021-08-25,164.959000,165.289001
4,MAZN,2021-08-26,165.800003,165.800003
...,...,...,...,...
751,ORCL,2022-08-15,79.110001,103.650002
752,ORCL,2022-08-16,79.519997,103.650002
753,ORCL,2022-08-17,79.250000,103.650002
754,ORCL,2022-08-18,79.320000,103.650002


In [100]:
query = """

select ticker, date, close as closing_rate, avg(close) over(partition by ticker order by date(Date)) as mavg
from ticker
order by ticker, date(Date)
"""

In [101]:
pd.read_sql(query, con=conn)

,Ticker,Date,closing_rate,mavg
0,MAZN,2021-08-20,159.997498,159.997498
1,MAZN,2021-08-23,163.293503,161.645500
2,MAZN,2021-08-24,165.289001,162.860001
3,MAZN,2021-08-25,164.959000,163.384750
4,MAZN,2021-08-26,165.800003,163.867801
...,...,...,...,...
751,ORCL,2022-08-15,79.110001,82.466895
752,ORCL,2022-08-16,79.519997,82.455060
753,ORCL,2022-08-17,79.250000,82.442240
754,ORCL,2022-08-18,79.320000,82.429801


### Previous day close

In [104]:
query = """

select ticker, date, close as closing_rate, lag(close, 1) over(partition by ticker order by date(Date)) as mavg
from ticker
order by ticker, date(Date)
"""

In [105]:
pd.read_sql(query, con=conn)

,Ticker,Date,closing_rate,mavg
0,MAZN,2021-08-20,159.997498,NaN
1,MAZN,2021-08-23,163.293503,159.997498
2,MAZN,2021-08-24,165.289001,163.293503
3,MAZN,2021-08-25,164.959000,165.289001
4,MAZN,2021-08-26,165.800003,164.959000
...,...,...,...,...
751,ORCL,2022-08-15,79.110001,79.150002
752,ORCL,2022-08-16,79.519997,79.110001
753,ORCL,2022-08-17,79.250000,79.519997
754,ORCL,2022-08-18,79.320000,79.250000


### Next day close with default value

In [110]:
query = """

select ticker, date, close as closing_rate, lead(close, 1,0) over(partition by ticker order by date(Date)) as mavg
from ticker
order by ticker, date(Date)
"""

In [111]:
pd.read_sql(query, con=conn)

,Ticker,Date,closing_rate,mavg
0,MAZN,2021-08-20,159.997498,163.293503
1,MAZN,2021-08-23,163.293503,165.289001
2,MAZN,2021-08-24,165.289001,164.959000
3,MAZN,2021-08-25,164.959000,165.800003
4,MAZN,2021-08-26,165.800003,167.481506
...,...,...,...,...
751,ORCL,2022-08-15,79.110001,79.519997
752,ORCL,2022-08-16,79.519997,79.250000
753,ORCL,2022-08-17,79.250000,79.320000
754,ORCL,2022-08-18,79.320000,78.660004
